Prompting Engineering

In [3]:
# 환경설정
import os
from dotenv import load_dotenv

load_dotenv() # 현재 경로의 .env 파일을 읽어 시스템 환경변수로 등록

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Char Completion

In [4]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {"type": "text", "text": "당신은 친절한 챗봇입니다."}
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "추운날 점심에는 뭘 먹으면 좋을까요?"}
            ]
        }

    ],
    response_format={"type": "text"},
    temperature=1.3,    # 대답 창의성(기본값 1) 0~2
    max_tokens= 2048,   # 응답 최대 토큰수
    top_p=1             # 사용할 상위누적확률
)

print(response.choices[0].message.content)



추운 날에는 몸을 따뜻하게 해주는 음식이 좋습니다. 예를 들어:

1. **김치찌개** - 따뜻한 국물이 매운 맛과 함께 몸을 데워줍니다.
2. **떡볶이** - 매콤한 고추장 소스가 추위 속에서 특히 맛있습니다.
3. **갈비탕** - 고기와 함께 진한 국물이 훈훈하게 몸속을 데워 줍니다.
4. **된장찌개** - 구수하고 진한 맛의 국물은 추운 날씨에 제격입니다.
5. **해물탕** - 다양한 해물과 매콤한 국물로 온기를 더할 수 있습니다.
6. **라면** - 간편하면서도 매운 라면이 좋은 선택입니다.

이와 같은 메뉴들이 추운 날씨에 맛있는 점심식사가 될 수 있습니다. 좋아하는 음식을 선택해 따뜻한 점심을 즐기세요!


### 패턴1: 페르소나 & Few-Shot(기사 제목 교정)

In [5]:
from openai import OpenAI

def corrent_title(query, temperature=0.3, max_tokens=2048, top_p=1):

    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    기사들이 숭고한 제목을 교정해주세요.
        ### 지시사항 ###
     - 기사의 제목이 명확하고 주제와 잘맞도록 수정해주세요
     - 비속어, 은어등은 제거하고 의미가 유지되도록 제목을 교정해주세요.
     - 간결하고 임팩트 있는 표현을 사용해주세요.
    """

    user_message = f"""
    다음 제목을 교정해주세요.
    제목 : {query}

    ### 출력 예시 ###
    # - 원래 제목 : [기사의 원래 제목]
    # - 교정 제목 :
    #       [교정된 제목1]
    #       [교정된 제목2]
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": system_instruction}
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_message}
                ]
            }
        ],
        response_format={"type": "text"},
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )

    return response.choices[0].message.content


print(corrent_title("추운날 점심에는 뭘 먹으면 좋을까요?", temperature=1.3))
print(corrent_title("추운날 점심에는 뭘 먹으면 좋을까요?", temperature=1.3))



    # - 원래 제목 : 추운날 점심에는 뭘 먹으면 좋을까요?
    # - 교정 제목 :
          추운 날 점심 추천 음식
          추위 속에서 즐기는 점심 메뉴 추천

    # - 원래 제목 : 추운날 점심에는 뭘 먹으면 좋을까요?
    # - 교정 제목 :
          추운 날씨에 좋은 점심 메뉴 추천
          추울 때 더욱 맛있는 따뜻한 점심메뉴


# 구조화된 출력 (structured Ouptput) - 영단어장 생성

In [6]:
from openai import OpenAI 
import json   # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def extract_eng_words(query, temperature=0.3, max_tokens=2048, top_p=1):

    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 영어팝송을 이용해 흥미롭게 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

    # 처리단계
    1. 주어진 가사에서 자주 사용되는 영어단어 5개를 랜덤으로 추출해주세요.
    2. 각 단어의 의미를 한글로 알려주세요.
    3. 단어별로 유사한 단어도 함께 소개해주세요.

    # 출력형식
    출력형식은 json 형식입니다.
    - 최상위 키는 "json_list" 여야 합니다.
    - "json_list"의 값은 단어별 json 객체들이 담긴 리스트여야 합니다.

    # 출력형식
    {
        "json_list" : [
            { "단어" : "yesterday",
              "의미" : "어제"
              "유사어" : [
                    {
                        "유사단어" : "today"   
                    }
                    {
                        "유사단어" : "tomorrow"
                    }
              ]   
            }
        ]
    }
    """

    user_message = f"""
    노래가사: {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": system_instruction}
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_message}
                ]
            }
        ],
        response_format={"type": "json_object"},
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )

    return json.loads(response.choices[0].message.content)


lyrics= """Something must've gone wrongin my brainGot your chemicalsall in my veinsFeeling all the highsfeeling all the painLet go of the waysI used to fall in laneNow I'm seeing rednot thinking straightBlurring all the linesyou intoxicate me
Just like nicotine
heroin morphine
Suddenly
I'm a fiend and you're all I need
All I need
Yeah you're all I need
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know
I'll never be the same
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will knew I'll never be the same
I'll never be the same
I'll never be the same
I'll never be the same
Sneaking in L.A.
when the lights are low
Off of one touch I could overdose
He said stop playing it safe
Girl I wanna see you lose control
Just like nicotine
heroin morphine
Suddenly
I'm a fiend and you're all I need
All I need
Ooh yeah you're all I need
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know
I'll never be the same
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know I'll never ever
ever be the same
I'll never be the same
I'll never be the same
I'll never be the same
You're in my blood
you're in my veins
you're in my head
You're in my blood
you're in my veins
you're in my head
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know
I'll never be the same
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will knew I'll never ever ever be the same """


result = extract_eng_words(lyrics, temperature=1)
words = result["json_list"]

# 파이썬 객체로 변환되어있기 때문에 for 반목문을 통해 각 요소를 순회하고 딕셔너리의 키를 이용해 원하는 데이터에 직접 접근 가능

for word_dict in words:
    print(f'단어: {word_dict["단어"]}')
    print(f'의미: {word_dict["의미"]}')


단어: babe
의미: 자기야
단어: need
의미: 필요하다
단어: move
의미: 움직이다
단어: feel
의미: 느끼다
단어: blame
의미: 비난하다


# 생각의 사슬(Chain-of-Thought) (냉털마스터)

* Reason + Act 기법으로 현재 상황에 대한 통찰 이후

In [ ]:
from openai import OpenAI 
import json   # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def extract_eng_words(query, temperature=0.3, max_tokens=2048, top_p=1):

    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    너는 냉장고에 있는 재료들을 활용하여 창의적이고 실용적인 저녁식사 아이디어를 제안하는 요리 전문가이다.
    네 역할을 사용자로부터 제공받은 재료목록을 분석하고, 이를 활용할 수 있는 요리 아이디어를 구상해서,
    조리방법을 단계별로 상세히 설명하는 것이다.
    """

    user_message = f"""
    1. 상황분석
    - 현재 가진 재료는 [사용자가 입력한 재료] 입니다.
    - 주재료인 닭고기와 다양한 채소들이 있습니다.
    - 이 재료들은 스튜, 볶음, 찜요리에 적합합니다.
    
    2. 행동계획
    - 가장 쉽게 만들수 있는 요리로 닭볶음탕을 제안합니다.
    - 각 요리에 필요한 재료와 조리도구를 확인합니다.
    - 부족한 재료가 있다면, 대체가능한 옵션을 제시합니다.
    - 조리 과정을 단계별로 상세히 설명합니다.
    - 맛을 더 향상시킬수 있는 팁과 주의사항 또한 제공합니다.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": system_instruction}
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_message}
                ]
            }
        ],
        response_format={"type": "json_object"},
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )

    return json.loads(response.choices[0].message.content)


lyrics= """Something must've gone wrongin my brainGot your chemicalsall in my veinsFeeling all the highsfeeling all the painLet go of the waysI used to fall in laneNow I'm seeing rednot thinking straightBlurring all the linesyou intoxicate me
Just like nicotine
heroin morphine
Suddenly
I'm a fiend and you're all I need
All I need
Yeah you're all I need
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know
I'll never be the same
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will knew I'll never be the same
I'll never be the same
I'll never be the same
I'll never be the same
Sneaking in L.A.
when the lights are low
Off of one touch I could overdose
He said stop playing it safe
Girl I wanna see you lose control
Just like nicotine
heroin morphine
Suddenly
I'm a fiend and you're all I need
All I need
Ooh yeah you're all I need
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know
I'll never be the same
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know I'll never ever
ever be the same
I'll never be the same
I'll never be the same
I'll never be the same
You're in my blood
you're in my veins
you're in my head
You're in my blood
you're in my veins
you're in my head
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will know
I'll never be the same
It's you babe
And I'm a sucker
for the way that you move
babe
And I could try to run
but it would be useless
You're to blame
Just one hit
you will knew I'll never ever ever be the same """


result = extract_eng_words(lyrics, temperature=1)
words = result["json_list"]

# 파이썬 객체로 변환되어있기 때문에 for 반목문을 통해 각 요소를 순회하고 딕셔너리의 키를 이용해 원하는 데이터에 직접 접근 가능

for word_dict in words:
    print(f'단어: {word_dict["단어"]}')
    print(f'의미: {word_dict["의미"]}')
